# Getting Started: Importing Dependencies and Data

In [1]:
# Dependencies
import pandas as pd
import os
from datetime import datetime

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, DateTime

# Imports method for temporarily binding to DBs
from sqlalchemy.orm import Session

In [2]:
# Read in CSVs with cleaned data
meas_csv_path = os.path.join('Resources', 'clean_hawaii_measurements.csv')
stations_csv_path = os.path.join('Resources', 'hawaii_stations.csv')

meas_df = pd.read_csv(meas_csv_path)
stations_df = pd.read_csv(stations_csv_path)

# Creating a SQLite Database

In [3]:
# Use the engine and connection string to create a database called hawaii.sqlite
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

In [4]:
# Use declarative_base and create ORM classes for each table.
# You will need a class for Measurement and for Station.
# Make sure to define your primary keys.

class Measurement(Base):
    __tablename__ = 'measurements'
    meas_id = Column(Integer, primary_key=True)
    station_id = Column(String(15))
    date = Column(DateTime)
    precip = Column(Float)
    tobs = Column(Float)

class Station(Base):
    __tablename__ = 'stations'
    station_id = Column(String(15), primary_key=True)
    name = Column(String(255))
    lat = Column(Float)
    lon = Column(Float)
    elev = Column(Float)

In [5]:
# Once you have your ORM classes defined, create the
# tables in the database using create_all.
Base.metadata.create_all(engine)

In [6]:
# Create a session.
session = Session(bind=engine)

In [7]:
# Add data from measurement CSV to database
meas_count = len(meas_df)
station_count = len(stations_df)
for i in range(0, meas_count-1):
    measurement = Measurement(meas_id=i,
                              station_id=meas_df.iloc[i]['station'],
                              date=datetime.strptime(meas_df.iloc[i]['date'], '%Y-%m-%d'),
                              precip=meas_df.iloc[i]['prcp'],
                              tobs=meas_df.iloc[i]['tobs'])
    session.add(measurement)

In [8]:
# Add data from stations CSV to database
station_count = len(stations_df)
for i in range(0, station_count-1):
    station = Station(station_id=stations_df.iloc[i]['station'],
                      name=stations_df.iloc[i]['name'],
                      lat=stations_df.iloc[i]['latitude'],
                      lon=stations_df.iloc[i]['longitude'],
                      elev=stations_df.iloc[i]['elevation'])
    session.add(station)

In [9]:
# Commit these objects to the database
session.commit()